In [ ]:
import numpy as np
import tensorflow as tf
import atomdnn
from atomdnn import data
from atomdnn import network
from atomdnn.data import *
from atomdnn.network import Network

# Creat Data object and reead inputdata from LAMMPS dump files

### read inputdata function:

### read_inputdata (fp_filename=None, der_filename=None, image_num=None,            read_der=TFAtomDNN.compute_force)

- **fp_filename**: fingerprints file path, use wildcard * in the name for a serials of files.

- **fp_filename**: derivatives file name, use wildcard * in the name for a serials of files.

- **image_num**: if not set, read all images

- **read_der**: set to true if read derivative data


It will generate input data in the form of tensorflow tensors, which can be accessed using keys:

- **input_dict [ 'fingerprints' ] [ i ] [ j ] [ k ]** gives the k-th fingerprint of j-th atom in i-th image.
    
- **input_dict [ 'atom_type' ] [ i ] [ j ]** gives the atom type of j-th atom in i-th image.
    
- **input_dict [ 'dGdr' ] [ i ] [ j ] [ k ] [ m ]** gives the derivative of k-th fingerprint in j-th derivative data blcok of i-th image w.r.t m-th cooridinate.
    
- **input_dict [ 'center_atom_id' ] [ i ] [ j ]** gives the center atom id in j-th derivative data block of i-th image.
    
- **input_dict [ 'neighbor_atom_id' ] [ i ] [ j ]** gives the neighbor atom id in j-th derivative data block of i-th image. Note that the neighbors could be ghost atoms.
    
- **input_dict [ 'neighbor_atom_coord' ] [ i ] [ j ] [ m ] [ 0 ]** gives the m-th coordiate of neighbor atom in j-th derivative block of i-th image. Note that the last dimension of the array is 1 which is added for matrix multiplication in neural network force_stress_layer. 

In [ ]:
grdata = Data()
fp_filename = '/workspace/data/group_share/data_graphene_ase/deform/dump_fp.*'
der_filename = '/workspace/data/group_share/data_graphene_ase/deform/dump_der.*'
grdata.read_inputdata(fp_filename=fp_filename,der_filename=der_filename)

In [ ]:
fp_filename = '/workspace/data/group_share/data_graphene_ase/move/dump_fp.*'
der_filename = '/workspace/data/group_share/data_graphene_ase/move/dump_der.*'
grdata.read_inputdata(fp_filename=fp_filename,der_filename=der_filename,append=True)

# Read output data genrated using LAMMPS

In [ ]:
output_file = '/workspace/data/group_share/data_graphene_ase/deform/lmp_output.dat'

grdata.read_outputdata(filename=output_file)

In [ ]:
output_file = '/workspace/data/group_share/data_graphene_ase/move/lmp_output.dat'

grdata.read_outputdata(filename=output_file,append=True)

# Create tensorflow dataset for training

In [ ]:
# check the data
grdata.check_data() 

In [ ]:
# this may take a while
import time
start_time = time.time()
tf_dataset = tf.data.Dataset.from_tensor_slices((grdata.input_dict,grdata.output_dict))
end_time = time.time()
print(end_time-start_time)

In [ ]:
tf.data.experimental.save(tf_dataset, '/workspace/data/group_share/graphene_tfdataset')

In [ ]:
newdataset = tf.data.experimental.load('/workspace/data/group_share/graphene_tfdataset')